In [1]:
import h2o
import pandas
import pprint
import operator
import matplotlib.pyplot as plt
from tabulate import tabulate

In [2]:
# Connect to a cluster
h2o.init()

H2O cluster uptime:,9 minutes 40 seconds 391 milliseconds
H2O cluster version:,3.1.0.99999
H2O cluster name:,Anqi
H2O cluster total nodes:,1
H2O cluster total memory:,1.78 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [3]:
# air_path = [h2o.locate("bigdata/laptop/airlines_all.05p.csv")]
# air_path = [h2o.locate("bigdata/laptop/flights-nyc/flights14.csv.zip")]
air_path = [h2o.locate("smalldata/airlines/allyears2k_headers.zip")]

# ----------

# 1- Load data - 1 row per flight.  Has columns showing the origin,
# destination, departure and arrival time, carrier information, and
# whether the flight was delayed.
print "Import and Parse airlines data"
data = h2o.import_frame(path=air_path)
data.describe()

Import and Parse airlines data

Parse Progress: [##################################################] 100%

Parsed 43,978 rows and 31 cols:



File1,C:\Users\Anqi\Documents\Work\h2o-3\smalldata/airlines/allyears2k_headers.zip


Rows: 43,978 Cols: 31

Chunk compression summary:



chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,10,5.376344,800 B,0.05040237
C0D,Constant Reals,23,12.365591,1.8 KB,0.115925446
CBS,Bits,2,1.0752689,2.0 KB,0.12720299
CX0,Sparse Bits,10,5.376344,1.9 KB,0.12474586
C1,1-Byte Integers,40,21.505377,287.8 KB,18.564957
C1N,1-Byte Integers (w/o NAs),19,10.215054,133.1 KB,8.58617
C1S,1-Byte Fractions,6,3.2258065,43.4 KB,2.8024976
C2,2-Byte Integers,76,40.860214,1.1 MB,69.628105



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
10.0.0.14:54321,1.5 MB,43978.0,6.0,186.0
mean,1.5 MB,43978.0,6.0,186.0
min,1.5 MB,43978.0,6.0,186.0
max,1.5 MB,43978.0,6.0,186.0
stddev,0 B,0.0,0.0,0.0
total,1.5 MB,43978.0,6.0,186.0



Column-by-Column Summary:



,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,17.0,14.0,-63.0,-16.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,9.0,3949.0,3500.0,475.0,437.0,402.0,475.0,473.0,131.0,133.0,3365.0,128.0,254.0,1.0,3.0,1.0,369.0,201.0,323.0,14.0,373.0,1.0,1.0
sigma,6.34436090171,1.87471137134,9.17579042586,1.90501311913,465.340899124,476.251139993,484.347487904,492.750434123,2.05121227084,777.404369164,1168.75931155,73.9744416606,73.40159463,69.6363295151,29.8402219624,26.4388090429,37.64411521,33.930070329,578.43800823,4.20197993986,9.9050857472,0.155193141358,0.182676305421,0.0497234872189,16.2057299045,4.41677989873,18.6197762215,0.403940182102,23.4875658741,0.496887288343,0.499377380318
zero_count,0,0,0,0,0,569,0,569,724,0,2,0,0,0,1514,6393,59,172,0,623,557,42892,81,43869,7344,8840,7388,8914,7140,19537,20887
missing_count,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0


In [4]:
# ----------

# 2- Data exploration and munging. Generate scatter plots 
# of various columns and plot fitted GLM model.

# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = h2o.glm(x = data[[x]], y = data[y], family = "gaussian")
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < float(max_points)/data.nrow()]
    df_py = h2o.as_list(df_subset)
    
    if(fit): h2o.remove(lr._key)

    # If x variable is string, generate box-and-whisker plot
    if(df_py[x].dtype == "object"):
        df_py.boxplot(column = y, by = x)
    # Otherwise, generate a scatter plot
    else:
        df_py.plot(x = x, y = y, kind = "scatter")
    
    if(fit):
        x_min = min(df_py[x])
        x_max = max(df_py[x])
        y_min = coeff["Intercept"] + coeff[x]*x_min
        y_max = coeff["Intercept"] + coeff[x]*x_max
        plt.plot([x_min, x_max], [y_min, y_max], "k-")
    plt.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "UniqueCarrier", "ArrDelay", max_points = 5000, fit = False)


glm Model Build Progress: [##################################################] 100%


In [5]:
# Group flights by month
aggregates = {"Month": ["nrow", 0, "all"], "Cancelled": ["sum", 1, "all"]}
bpd = h2o.group_by(data, cols=["Month"], aggregates=aggregates)
bpd.show()
bpd.describe()
bpd.dim()

# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

Displaying 2 row(s):


Row ID,Month,Cancelled,Month
1,[10.0],[19990.0],[1999.0]
2,[1.0],[41979.0],[41979.0]


Rows: 2 Cols: 3

Chunk compression summary:



chunk_type,chunk_name,count,count_percentage,size,size_percentage
C1N,1-Byte Integers (w/o NAs),1,33.333336,70 B,28.455284
C2S,2-Byte Fractions,2,66.66667,176 B,71.544716



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
10.0.0.14:54321,246 B,2.0,1.0,3.0
mean,246 B,2.0,1.0,3.0
min,246 B,2.0,1.0,3.0
max,246 B,2.0,1.0,3.0
stddev,0 B,0.0,0.0,0.0
total,246 B,2.0,1.0,3.0



Column-by-Column Summary:



,Month,Cancelled,Month
type,int,int,int
mins,1.0,19990.0,1999.0
maxs,10.0,41979.0,41979.0
sigma,6.36396103068,15548.5710115,28270.1291118
zero_count,0,0,0
missing_count,0,0,0


In [6]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100
mins1 = data["CRSArrTime"] % 100
arrTime = hour1*60 + mins1

hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2

# TODO: Replace this once list comprehension is supported. See PUBDEV-1286.
# data["TravelTime"] = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = h2o.ifelse((arrTime-depTime) > 0, (arrTime-depTime), None)[0]
scatter_plot(data, "Distance", "TravelTime")


glm Model Build Progress: [##################################################] 100%


In [7]:
# Impute missing travel times and re-plot
h2o.impute(data = data, column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")


glm Model Build Progress: [##################################################] 100%


In [8]:
# ----------
# 3- Fit a model on train; using test as validation

# Create test/train split
s = data["Year"].runif()
train = data[s <= 0.75]
test  = data[s > 0.75]

# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

# Simple GLM - Predict Delays
data_glm = h2o.glm(x           =train[myX],
                   y           =train[myY],
                   validation_x=test [myX],
                   validation_y=test [myY],
                   family      ="binomial",
                   standardize =True)

# Simple GBM
data_gbm = h2o.gbm(x              =train[myX],
                   y              =train[myY],
                   validation_x   =test [myX],
                   validation_y   =test [myY],
                   balance_classes=True,
                   ntrees         =3,
                   max_depth      =1,
                   distribution   ="bernoulli",
                   learn_rate     =0.1,
                   min_rows       =2)

# Complex GBM
data_gbm2 = h2o.gbm(x              =train[myX],
                    y              =train[myY],
                    validation_x   =test [myX],
                    validation_y   =test [myY],
                    balance_classes=True,
                    ntrees         =50,
                    max_depth      =5,
                    distribution   ="bernoulli",
                    learn_rate     =0.1,
                    min_rows       =2)

# Simple Random Forest
data_rf = h2o.random_forest(x              =train[myX],
                            y              =train[myY],
                            validation_x   =test [myX],
                            validation_y   =test [myY],
                            ntrees         =5,
                            max_depth      =2,
                            balance_classes=True)

# Complex Random Forest
data_rf2 = h2o.random_forest(x              =train[myX], 
                             y              =train[myY],
                             validation_x   =test [myX],
                             validation_y   =test [myY],
                             ntrees         =10,
                             max_depth      =5,
                             balance_classes=True)

# Deep Learning with 5 epochs
data_dl = h2o.deeplearning(x                   =train[myX],
                           y                   =train[myY],
                           validation_x        =test [myX],
                           validation_y        =test [myY],
                           hidden              =[10,10],
                           epochs              =5,
                           variable_importances=True,
                           balance_classes     =True,
                           loss                ="Automatic")


glm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

deeplearning Model Build Progress: [##################################################] 100%


In [19]:
# Variable importances from each algorithm
# Calculate magnitude of normalized GLM coefficients
glm_varimp = data_glm.coef_norm()
for k,v in glm_varimp.iteritems():
    glm_varimp[k] = abs(glm_varimp[k])
    
# Sort in descending order by magnitude
glm_sorted = sorted(glm_varimp.items(), key = operator.itemgetter(1), reverse = True)
table = tabulate(glm_sorted, headers = ["Predictor", "Normalized Coefficient"], tablefmt = "orgtbl")
print "Variable Importances:\n\n" + table

data_gbm.varimp()
data_rf.varimp()

Variable Importances:

| Predictor        |   Normalized Coefficient |
|------------------+--------------------------|
| Year.2008        |              2.20645     |
| Dest.LYH         |              1.66734     |
| Year.2003        |              1.527       |
| Year.2007        |              1.45986     |
| Origin.HPN       |              1.38768     |
| Origin.MDW       |              1.34977     |
| Origin.LIH       |              1.33007     |
| Origin.LEX       |              1.31243     |
| Origin.CAE       |              1.17961     |
| UniqueCarrier.HP |              1.16061     |
| Origin.TLH       |              1.11197     |
| Origin.HNL       |              1.10426     |
| Year.2001        |              1.04418     |
| Origin.ERI       |              1.04216     |
| Origin.MLB       |              1.01101     |
| Origin.OGG       |              0.98171     |
| Origin.GRR       |              0.968939    |
| Year.2002        |              0.942916    |
| Origin.MYR     

variable,relative_importance,scaled_importance,percentage
Year,1234.6340332,1.0,1.0
Origin,0.0,0.0,0.0
Dest,0.0,0.0,0.0
UniqueCarrier,0.0,0.0,0.0
DayOfWeek,0.0,0.0,0.0
Month,0.0,0.0,0.0
Distance,0.0,0.0,0.0
FlightNum,0.0,0.0,0.0



Variable Importances:



variable,relative_importance,scaled_importance,percentage
Year,1895.45153809,1.0,0.437243955524
Origin,1551.8223877,0.81870855388,0.35797536652
Dest,465.785095215,0.245738329815,0.107447599352
FlightNum,260.802764893,0.137594003145,0.0601621461915
DayOfWeek,55.2715072632,0.0291600740787,0.0127500661336
UniqueCarrier,53.0120201111,0.0279680166155,0.0122288462131
Distance,52.8523788452,0.0278837932721,0.0121920200653
Month,0.0,0.0,0.0


In [20]:
# Model performance of GBM model on test data
data_gbm2.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.206072998286
R^2: 0.173530324799
LogLoss: 0.598709245809
AUC: 0.740860130858
Gini: 0.481720261716

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.390111773837:



,NO,YES,Error,Rate
NO,2418.0,2820.0,0.5384,(2820.0/5238.0)
YES,886.0,4929.0,0.1524,(886.0/5815.0)
Total,3304.0,7749.0,0.6908,(0.6908/11053.0)



Maximum Metrics:



metric,threshold,value,idx
max f1,0.390111773837,0.726776762017,271.0
max f2,0.203821606737,0.848781779661,366.0
max f0point5,0.584877735232,0.701223357296,170.0
max accuracy,0.485219911311,0.678277390754,220.0
max precision,0.953543170817,1.0,0.0
max absolute_MCC,0.584877735232,0.35864900149,170.0
max min_per_class_accuracy,0.507749761479,0.675448644521,209.0
